In [1]:
import datetime
from heapq import nlargest
from operator import itemgetter
import os
import time
import math
from collections import defaultdict

def apk(actual, predicted, k=3):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def prep_xy(x, y, virtual_range_x, virtual_range_y, real_range_x, real_range_y):    
    ix = math.floor(virtual_range_x*x/real_range_x)
    if ix < 0:
        ix = 0
    if ix >= virtual_range_x:
        ix = virtual_range_x-1

    iy = math.floor(virtual_range_y*y/real_range_y)
    if iy < 0:
        iy = 0
    if iy >= virtual_range_y:
        iy = virtual_range_y-1

    return ix, iy

xpy = dict()
for x in [-2,2]:
    for y in [-2,2]:
        for p in [-2.2]:
            xpy[(x,y,p) = 1]

In [2]:
def calculate_region(start_x, end_x, start_y, end_y, margin_weight, result_name):
    start_time = time.time()
    print('Preparing arrays...', flush = True)
    f = open("../train.csv", "r")
    f.readline()
    total = 0
    virtual_range_x = 290 #580 #870 #290 #500
    virtual_range_y = 725 #1450 #2175 #725 #1000
    real_range_x = end_x - start_x
    real_range_y = end_y - start_y
    train_extra_margin = 0.05
    train_start_x = max(start_x - train_extra_margin, 0)
    train_end_x = min(end_x + train_extra_margin, 10)
    train_start_y = max(start_y - train_extra_margin, 0)
    train_end_y = min(end_y + train_extra_margin, 10)
    day_partition_size = 4
    
    split_t = math.floor((1.0 - 0.125) * 786239)
    out_of_business_time = 0.125
    split_test_out_of_business = math.floor((1.0 - 0.125 - out_of_business_time) * 786239)
    split_submit_out_of_business = math.floor((1.0 - out_of_business_time) * 786239)
    test_arr = []
    
    grid = defaultdict(lambda: defaultdict(int))
    grid_valid = defaultdict(lambda: defaultdict(int))
    submit_out_of_business = dict()
    test_out_of_business = dict()
    grid_sorted = dict()
    grid_sorted_valid = dict()
    
    train_samples = 0
    test_samples = 0
    while 1:
        line = f.readline().strip()
        total += 1
        
        if line == '':
            break
        
        if(total % 6000000 == 0):
            print('Checkpoint ...',total, flush = True)
        
        arr = line.split(",")
        #row_id = arr[0]
        x = float(arr[1])
        y = float(arr[2])
        if(x < train_start_x or x > train_end_x or y < train_start_y or y > train_end_y):
            continue
        
        #accuracy = int(arr[3])
        time1 = int(arr[4])
        place_id = arr[5]
        partition_of_day = math.floor((time1 + 120) / (day_partition_size*60)) % int(24/day_partition_size)
        log_month = math.log10(3+((time1 + 120.0) / (60 * 24 * 30)))
        
        
        ix, iy = prep_xy(x - start_x, y - start_y, virtual_range_x, virtual_range_y, real_range_x, real_range_y)
        grid[(ix, iy, partition_of_day)][place_id] += (1) * log_month
        grid[(ix, iy + 1, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix, iy - 1, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix + 1, iy, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix + 1, iy + 1, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix + 1, iy - 1, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix - 1, iy, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix - 1, iy + 1, partition_of_day)][place_id] += (1) * log_month 
        grid[(ix - 1, iy - 1, partition_of_day)][place_id] += (1) * log_month 
        
        # margin weight
        grid[(ix, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 1, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 1, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 1, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 1, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy + 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy - 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy + 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy - 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
        # end margin weight
        
        grid[(ix, iy)][place_id] += 1 * log_month
        grid[(ix, iy + 1)][place_id] += 1 * log_month
        grid[(ix, iy - 1)][place_id] += 1 * log_month
        grid[(ix + 1, iy)][place_id] += 1 * log_month
        grid[(ix + 1, iy + 1)][place_id] += 1 * log_month
        grid[(ix + 1, iy - 1)][place_id] += 1 * log_month
        grid[(ix - 1, iy)][place_id] += 1 * log_month
        grid[(ix - 1, iy + 1)][place_id] += 1 * log_month
        grid[(ix - 1, iy - 1)][place_id] += 1 * log_month
        
        # margin weight
        grid[(ix, iy + 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 1, iy + 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 1, iy + 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix, iy - 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 1, iy - 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 1, iy - 2)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy + 1)][place_id] += (1) * log_month * margin_weight
        grid[(ix + 2, iy - 1)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy + 1)][place_id] += (1) * log_month * margin_weight
        grid[(ix - 2, iy - 1)][place_id] += (1) * log_month * margin_weight
        # end margin weight
        
        if time1 < split_t:
            grid_valid[(ix, iy, partition_of_day)][place_id] += (1) * log_month
            grid_valid[(ix, iy + 1, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix, iy - 1, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix + 1, iy, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix + 1, iy + 1, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix + 1, iy - 1, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix - 1, iy, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix - 1, iy + 1, partition_of_day)][place_id] += (1) * log_month 
            grid_valid[(ix - 1, iy - 1, partition_of_day)][place_id] += (1) * log_month 
            
            # margin weight
            grid_valid[(ix, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 1, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 1, iy + 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 1, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 1, iy - 2, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy + 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy - 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy + 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy - 1, partition_of_day)][place_id] += (1) * log_month * margin_weight
            # end margin weight
            
            grid_valid[(ix, iy)][place_id] += 1 * log_month
            grid_valid[(ix, iy + 1)][place_id] += 1 * log_month
            grid_valid[(ix, iy - 1)][place_id] += 1 * log_month
            grid_valid[(ix + 1, iy)][place_id] += 1 * log_month
            grid_valid[(ix + 1, iy + 1)][place_id] += 1 * log_month
            grid_valid[(ix + 1, iy - 1)][place_id] += 1 * log_month
            grid_valid[(ix - 1, iy)][place_id] += 1 * log_month
            grid_valid[(ix - 1, iy + 1)][place_id] += 1 * log_month
            grid_valid[(ix - 1, iy - 1)][place_id] += 1 * log_month
            
            # margin weight
            grid_valid[(ix, iy + 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 1, iy + 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 1, iy + 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix, iy - 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 1, iy - 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 1, iy - 2)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy + 1)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix + 2, iy - 1)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy + 1)][place_id] += (1) * log_month * margin_weight
            grid_valid[(ix - 2, iy - 1)][place_id] += (1) * log_month * margin_weight
            # end margin weight
            
            train_samples += 1
            if time1 >= split_test_out_of_business:
                test_out_of_business[place_id] = 1
        else:
            test_arr.append(arr)
            test_samples += 1
        
        if time1 >= split_submit_out_of_business:
            submit_out_of_business[place_id] = 1
    
    f.close()
    
    print('Sorting arrays...', flush = True)
    for el in grid:
        grid_sorted[el] = nlargest(3, sorted(grid[el].items()), key=itemgetter(1))
    for el in grid_valid:
        grid_sorted_valid[el] = nlargest(3, sorted(grid_valid[el].items()), key=itemgetter(1))
      
    print('Run validation...', flush = True)
    total = 0
    score = 0.0
    score_num = 0
    
    for arr in test_arr:
        total += 1
        #row_id = arr[0]
        x = float(arr[1])
        y = float(arr[2])
        if(x < start_x or x > end_x or y < start_y or y > end_y):
            continue
        #accuracy = int(arr[3])
        time1 = int(arr[4])
        place_id = arr[5]
        partition_of_day = math.floor((time1 + 120) / (day_partition_size*60)) % int(24/day_partition_size)
        filled = []
        ix, iy = prep_xy(x - start_x, y - start_y, virtual_range_x, virtual_range_y, real_range_x, real_range_y)
        s1 = (ix, iy, partition_of_day)
        s2 = (ix, iy)
        if len(filled) < 3 and s1 in grid_sorted_valid:
            topitems = grid_sorted_valid[s1]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                if topitems[i][0] in test_out_of_business:
                    filled.append(topitems[i][0])
        if len(filled) < 3 and s2 in grid_sorted_valid:
            topitems = grid_sorted_valid[s2]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                if topitems[i][0] in test_out_of_business:
                    filled.append(topitems[i][0])
        if len(filled) < 3 and s1 in grid_sorted_valid:
            topitems = grid_sorted_valid[s1]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                filled.append(topitems[i][0])
        if len(filled) < 3 and s2 in grid_sorted_valid:
            topitems = grid_sorted_valid[s2]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                filled.append(topitems[i][0])
        score += apk([place_id], filled, 3)
            
        score_num += 1
    
    score /= score_num
    print('Predicted score: {}'.format(score), flush = True)
    print('Train samples: ', train_samples, flush = True)
    print('Test samples: ', test_samples, flush = True)
    
    print('Generate submission...')
    out = open(result_name, "w")
    f = open("../test.csv", "r")
    f.readline()
    total = 0
    count_empty0 = 0
    count_empty1 = 0
    count_empty2 = 0
    out.write("row_id,place_id\n")
    
    while 1:
        line = f.readline().strip()
        total += 1
        
        if line == '':
            break
        
        arr = line.split(",")
        row_id = arr[0]
        x = float(arr[1])
        y = float(arr[2])
        
        if(x < start_x or x > end_x or y < start_y or y > end_y):
            continue
        
        time1 = int(arr[4])
        partition_of_day = math.floor((time1 + 120) / (day_partition_size*60)) % int(24/day_partition_size)
        
        out.write(str(row_id) + ',')
        filled = []
        
        ix, iy = prep_xy(x - start_x, y - start_y, virtual_range_x, virtual_range_y, real_range_x, real_range_y)
        
        s1 = (ix, iy, partition_of_day)
        s2 = (ix, iy)
        if len(filled) < 3 and s1 in grid_sorted:
            topitems = grid_sorted[s1]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                if topitems[i][0] in submit_out_of_business:
                    out.write(' ' + topitems[i][0])
                    filled.append(topitems[i][0])
        if len(filled) < 3 and s2 in grid_sorted:
            topitems = grid_sorted[s2]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                if topitems[i][0] in submit_out_of_business:
                    out.write(' ' + topitems[i][0])
                    filled.append(topitems[i][0])
        if len(filled) < 3 and s1 in grid_sorted:
            topitems = grid_sorted[s1]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                out.write(' ' + topitems[i][0])
                filled.append(topitems[i][0])
        if len(filled) < 3 and s2 in grid_sorted:
            topitems = grid_sorted[s2]
            for i in range(len(topitems)):
                if topitems[i][0] in filled:
                    continue
                if len(filled) == 3:
                    break
                out.write(' ' + topitems[i][0])
                filled.append(topitems[i][0])
        
        if len(filled) == 0:
            count_empty0 += 1
        if len(filled) == 1:
            count_empty1 += 1
        if len(filled) == 2:
            count_empty2 += 1
        out.write("\n")
    
    print('Empty0 cases:', str(count_empty0), flush = True)
    print('Empty1 cases:', str(count_empty1), flush = True)
    print('Empty2 cases:', str(count_empty2), flush = True)
    out.close()
    f.close()
    
    print("Elapsed time overall: %s seconds" % (time.time() - start_time), flush = True)
    return score

In [3]:
total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    0, 'm0result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  0, 'm0result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   0, 'm0result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    0, 'm0result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  0, 'm0result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   0, 'm0result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    0, 'm0result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  0, 'm0result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   0, 'm0result9.csv')
print("Final score ", total_score / 9)


total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    0.2, 'm2result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  0.2, 'm2result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   0.2, 'm2result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    0.2, 'm2result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  0.2, 'm2result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   0.2, 'm2result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    0.2, 'm2result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  0.2, 'm2result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   0.2, 'm2result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    0.4, 'm4result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  0.4, 'm4result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   0.4, 'm4result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    0.4, 'm4result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  0.4, 'm4result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   0.4, 'm4result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    0.4, 'm4result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  0.4, 'm4result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   0.4, 'm4result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    0.6, 'm6result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  0.6, 'm6result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   0.6, 'm6result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    0.6, 'm6result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  0.6, 'm6result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   0.6, 'm6result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    0.6, 'm6result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  0.6, 'm6result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   0.6, 'm6result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    0.8, 'm8result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  0.8, 'm8result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   0.8, 'm8result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    0.8, 'm8result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  0.8, 'm8result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   0.8, 'm8result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    0.8, 'm8result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  0.8, 'm8result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   0.8, 'm8result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1.0, 'm10result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1.0, 'm10result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1.0, 'm10result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1.0, 'm10result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1.0, 'm10result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1.0, 'm10result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1.0, 'm10result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1.0, 'm10result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1.0, 'm10result9.csv')
print("Final score ", total_score / 9)

Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.49453302197290916
Train samples:  2787616
Test samples:  491584
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2 cases: 0
Elapsed time overall: 422.27791452407837 seconds
Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.497150288796416
Train samples:  2761481
Test samples:  491479
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2 cases: 0
Elapsed time overall: 432.8363597393036 seconds
Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.49172575194822693
Train samples:  2846023
Test samples:  519532
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2

0	Line 136: Final score  0.491932530621264 <br />
2	Line 272: Final score  0.4991593920694491 <br />
4	Line 408: Final score  0.5015781278291418 <br />
6	Line 544: Final score  0.5024910249246728 <br />
8	Line 680: Final score  0.5021992455636582 <br />
1	Line 816: Final score  0.5018204608750194 <br />

Submit 6: 

In [3]:
total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1, 'baseresult1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1, 'baseresult2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1, 'baseresult3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1, 'baseresult4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1, 'baseresult5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1, 'baseresult6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1, 'baseresult7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1, 'baseresult8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1, 'baseresult9.csv')
print("Final score ", total_score / 9)


total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1.2, 'p2result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1.2, 'p2result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1.2, 'p2result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1.2, 'p2result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1.2, 'p2result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1.2, 'p2result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1.2, 'p2result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1.2, 'p2result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1.2, 'p2result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1.4, 'p4result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1.4, 'p4result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1.4, 'p4result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1.4, 'p4result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1.4, 'p4result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1.4, 'p4result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1.4, 'p4result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1.4, 'p4result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1.4, 'p4result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1.6, 'p6result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1.6, 'p6result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1.6, 'p6result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1.6, 'p6result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1.6, 'p6result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1.6, 'p6result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1.6, 'p6result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1.6, 'p6result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1.6, 'p6result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    1.8, 'p8result1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  1.8, 'p8result2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   1.8, 'p8result3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    1.8, 'p8result4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  1.8, 'p8result5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   1.8, 'p8result6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    1.8, 'p8result7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  1.8, 'p8result8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   1.8, 'p8result9.csv')
print("Final score ", total_score / 9)

total_score = 0
total_score += calculate_region(0, 3.3,   0, 3.3,    2.0, '2presult1.csv')
total_score += calculate_region(0, 3.3,   3.3, 6.6,  2.0, '2presult2.csv')
total_score += calculate_region(0, 3.3,   6.6, 10,   2.0, '2presult3.csv')
total_score += calculate_region(3.3, 6.6, 0, 3.3,    2.0, '2presult4.csv')
total_score += calculate_region(3.3, 6.6, 3.3, 6.6,  2.0, '2presult5.csv')
total_score += calculate_region(3.3, 6.6, 6.6, 10,   2.0, '2presult6.csv')
total_score += calculate_region(6.6, 10,  0, 3.3,    2.0, '2presult7.csv')
total_score += calculate_region(6.6, 10,  3.3, 6.6,  2.0, '2presult8.csv')
total_score += calculate_region(6.6, 10,  6.6, 10,   2.0, '2presult9.csv')
print("Final score ", total_score / 9)

Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.4946675303152222
Train samples:  2787616
Test samples:  491584
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2 cases: 0
Elapsed time overall: 263.12951254844666 seconds
Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.4972604379617621
Train samples:  2761481
Test samples:  491479
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2 cases: 1
Elapsed time overall: 263.793833732605 seconds
Preparing arrays...
Checkpoint ... 6000000
Checkpoint ... 12000000
Checkpoint ... 18000000
Checkpoint ... 24000000
Sorting arrays...
Run validation...
Predicted score: 0.4918942843443762
Train samples:  2846023
Test samples:  519532
Generate submission...
Empty0 cases: 0
Empty1 cases: 0
Empty2 c

Summary:<br />
Weight is not helpful in 3*3 square<br />
Line 136: Final score  0.49205710365765004<br />
Line 272: Final score  0.49171579042542124<br />
Line 408: Final score  0.49148144785567927<br />
Line 544: Final score  0.4910832199017322<br />
Line 680: Final score  0.4905506838262989<br />
Line 816: Final score  0.4898162315000374<br />
